In [1]:
import os
import requests
import json
from pprint import pprint
from api_key import api_key
import pandas as pd

In [2]:
# File to Load 
michelin_df = pd.read_csv("Resources/Michelin-Starred Restaurants.csv", encoding='utf-8')
michelin_df.head()

,Unnamed: 0,name,address,city,state,description,star,full_address,postal_code
0,1,Sushi Inoue,381 Lenox Ave,New York,NY,ONE STAR for this Japanese restaurant in Harlem.,1,"381 Lenox Ave, New York, NY 10027",10027
1,2,Dovetail,103 W 77th St,New York,NY,John Fraser's UWS fine dining restaurant has O...,1,"103 W 77th St, New York, NY 10024",10024
2,3,Cafe Boulud,20 E 76th St,New York,NY,Daniel Boulud's beloved Upper East Side restau...,1,"20 E 76th St, New York, NY 10021",10021
3,4,Jean-Georges,1 Central Park West,New York,NY,Jean-Georges Vongerichten's flagship restauran...,2,"1 Central Park West, New York, NY 10023",10023
4,5,Masa,10 Columbus Cir,New York,NY,Masa Takayama's pricey sushi restaurant has TH...,3,"10 Columbus Cir, New York, NY 10019",10019


In [3]:
"""
Yelp Fusion API code sample.
This program demonstrates the capability of the Yelp Fusion API
by using the Search API to query for businesses by a search term and location,
and the Business API to query additional information about the top result
from the search query.
Please refer to http://www.yelp.com/developers/v3/documentation for the API
documentation.
This program requires the Python requests library, which you can install via:
`pip install -r requirements.txt`.
Sample usage of the program:
`python sample.py --term="bars" --location="San Francisco, CA"`
"""
from __future__ import print_function

import argparse
import json
import pprint
import requests
import sys
import urllib


# This client code can run on Python 2.x or 3.x.  Your imports can be
# simpler if you only need one of those.
try:
    # For Python 3.0 and later
    from urllib.error import HTTPError
    from urllib.parse import quote
    from urllib.parse import urlencode
except ImportError:
    # Fall back to Python 2's urllib2 and urllib
    from urllib2 import HTTPError
    from urllib import quote
    from urllib import urlencode


# Yelp Fusion no longer uses OAuth as of December 7, 2017.
# You no longer need to provide Client ID to fetch Data
# It now uses private keys to authenticate requests (API Key)
# You can find it on
# https://www.yelp.com/developers/v3/manage_app
API_KEY= None 


# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.


# Defaults for our simple example.
DEFAULT_TERM = 'dinner'
DEFAULT_LOCATION = 'San Francisco, CA'
SEARCH_LIMIT = 3


def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()


def search(api_key, term, location):
    """Query the Search API by a search term and location.
    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT,
        'attributes': 'reservation'
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)


def get_business(api_key, business_id):
    """Query the Business API by a business ID.
    Args:
        business_id (str): The ID of the business to query.
    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, api_key)


def query_api(term, location):
    """Queries the API by the input values from the user.
    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    response = search(API_KEY, term, location)

    businesses = response.get('businesses')

    if not businesses:
        print(u'No businesses for {0} in {1} found.'.format(term, location))
        return

    business_id = businesses[0]['id']

    print(u'{0} businesses found, querying business info ' \
        'for the top result "{1}" ...'.format(
            len(businesses), business_id))
    response = get_business(API_KEY, business_id)

    print(u'Result for business "{0}" found:'.format(business_id))
    pprint.pprint(response, indent=2)


def main():
    parser = argparse.ArgumentParser()

    parser.add_argument('-q', '--term', dest='term', default=DEFAULT_TERM,
                        type=str, help='Search term (default: %(default)s)')
    parser.add_argument('-l', '--location', dest='location',
                        default=DEFAULT_LOCATION, type=str,
                        help='Search location (default: %(default)s)')

    input_values = parser.parse_args()

    try:
        query_api(input_values.term, input_values.location)
    except HTTPError as error:
        sys.exit(
            'Encountered HTTP error {0} on {1}:\n {2}\nAbort program.'.format(
                error.code,
                error.url,
                error.read(),
            )
        )



In [4]:
results = search(api_key, 'Sushi Inoue', location='new york')
results['businesses'][0]

Querying https://api.yelp.com/v3/businesses/search ...


{'id': '6qzNc9Z9DHYYleKqrW_K3g',
 'alias': 'takashi-new-york',
 'name': 'Takashi',
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/dD_Aej4BzHix1fSXIEdBnw/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/takashi-new-york?adjust_creative=WSL6MviQWcNWEEaCgeUQAA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=WSL6MviQWcNWEEaCgeUQAA',
 'review_count': 977,
 'categories': [{'alias': 'japanese', 'title': 'Japanese'},
  {'alias': 'korean', 'title': 'Korean'},
  {'alias': 'bbq', 'title': 'Barbeque'}],
 'rating': 4.0,
 'coordinates': {'latitude': 40.7313524, 'longitude': -74.0065048},
 'transactions': ['restaurant_reservation'],
 'price': '$$$',
 'location': {'address1': '456 Hudson St',
  'address2': '',
  'address3': '',
  'city': 'New York',
  'zip_code': '10014',
  'country': 'US',
  'state': 'NY',
  'display_address': ['456 Hudson St', 'New York, NY 10014']},
 'phone': '+12124142929',
 'display_phone': '(212) 414-2929',
 'distance': 3067.5080525117

In [5]:
results['businesses'][0]['coordinates']['latitude']

40.7313524

In [6]:
restaurants = michelin_df['name']
restaurants

0         Sushi Inoue
1            Dovetail
2         Cafe Boulud
3        Jean-Georges
4                Masa
           ...       
67     The River Café
68            La Vara
69          The Finch
70             Per Se
71              Agern
Name: name, Length: 72, dtype: object

In [7]:
# for i in range(len(restaurants)):
#     print(f'{restaurants[i]}')

In [8]:
results = search(api_key, 'Dovetail', location='new york')
latitude = results['businesses'][0]['coordinates']['latitude']

Querying https://api.yelp.com/v3/businesses/search ...


IndexError: list index out of range

In [9]:
lat = [];
lon = [];

for i in range(len(restaurants)):
    try: 
        results = search(api_key, f'{restaurants[i]}', location='new york')
        print('===========')
        print(f'{restaurants[i]}')
        latitude = results['businesses'][0]['coordinates']['latitude']
        longitude = results['businesses'][0]['coordinates']['longitude']
        print(f'latitude is {latitude}')
        print(f'longitude is {longitude}')
        lat.append(latitude)
        lon.append(longitude)
        print('===========')
    except IndexError:
        print('===========')
        print(f'{restaurants[i]} has a different response')
        latitude2 = results['region']['center']['latitude']
        longitude2 = results['region']['center']['longitude']
        print(f'latitude is {latitude2}')
        print(f'longitude is {longitude2}')
        lat.append(latitude2)
        lon.append(longitude2)
        print('===========')
        continue



Querying https://api.yelp.com/v3/businesses/search ...
 Sushi Inoue
latitude is 40.7313524
longitude is -74.0065048
Querying https://api.yelp.com/v3/businesses/search ...
 Dovetail
 Dovetail has a different response
latitude is 40.70544486444615
longitude is -73.99429321289062
Querying https://api.yelp.com/v3/businesses/search ...
 Cafe Boulud
latitude is 40.7136187
longitude is -73.9888503
Querying https://api.yelp.com/v3/businesses/search ...
 Jean-Georges
latitude is 40.7327
longitude is -73.99766
Querying https://api.yelp.com/v3/businesses/search ...
 Masa
latitude is 40.75851
longitude is -73.98322
Querying https://api.yelp.com/v3/businesses/search ...
 Marea
latitude is 40.7763330374196
longitude is -73.9105868339539
Querying https://api.yelp.com/v3/businesses/search ...
 Daniel
latitude is 40.73715
longitude is -73.98789
Querying https://api.yelp.com/v3/businesses/search ...
 Torishin
latitude is 40.74545
longitude is -73.98499
Querying https://api.yelp.com/v3/businesses/search 

 Uncle Boons
latitude is 40.7303859
longitude is -73.9860613
Querying https://api.yelp.com/v3/businesses/search ...
 Contra
latitude is 40.09073
longitude is -74.05147
Querying https://api.yelp.com/v3/businesses/search ...
 Bâtard
latitude is 40.820352
longitude is -73.409301
Querying https://api.yelp.com/v3/businesses/search ...
 Jungsik
latitude is 40.73684
longitude is -73.99069
Querying https://api.yelp.com/v3/businesses/search ...
 Delaware and Hudson
latitude is 40.71702
longitude is -73.959077
Querying https://api.yelp.com/v3/businesses/search ...
 Atera
 Atera has a different response
latitude is 40.70544486444615
longitude is -73.99429321289062
Querying https://api.yelp.com/v3/businesses/search ...
 Aska
latitude is 40.7525286099337
longitude is -73.9780961338684
Querying https://api.yelp.com/v3/businesses/search ...
 L'Appart
 L'Appart has a different response
latitude is 40.70544486444615
longitude is -73.99429321289062
Querying https://api.yelp.com/v3/businesses/search ...


In [10]:
len(lat)

72

In [11]:
names = []

for i in range(len(restaurants)):
    resto = restaurants[i]
    names.append(resto)

In [12]:
address = []

for i in range(len(restaurants)):
    location = michelin_df['full_address'][i]
    address.append(location)

# address

In [13]:
data_set = {"restaurants": [names],
           "latitude": [lat], 
           "longitude": [lon],
           "address": [address]}
# data_set

In [14]:
json_dump = json.dumps(data_set)
print(json_dump)

{"restaurants": [[" Sushi Inoue", " Dovetail", " Cafe Boulud", " Jean-Georges", " Masa", " Marea", " Daniel", " Torishin", " Le Bernardin", " The Modern", " Aquavit", " Caviar Russe", " Satsuki (Suzuki\u2019s sushi bar)", " Chef's Table at Brooklyn Fare", " Aureole", " Gabriel Kreuther", " Sushi Ginza Onodera", " Sushi Amane", " Sushi Yasuda", " Ai Fiori", " Caf\u00e9 China", " Kajitsu", " Tempura Matsui", " Ushiwakamaru", " The Breslin", " The NoMad Restaurant", " Del Posto", " Casa Enrique", " Junoon", " La Sirena", " Eleven Madison Park", " The Clocktower", " Cote Korean Steakhouse", " Rouge Tomate Chelsea", " Gunter Seeger NY", " ALDEA", " Gramercy Tavern", " Casa Mono", " Walls\u00e9", " Gotham Bar and Grill", " Nix", " Babbo", " Blue Hill", " Kanoyama", " Minetta Tavern", " Sushi Zo", " Carbone", " ZZ's Clam Bar", " Jewel Bako", " Kyo Ya", " Momofuku Ko", " Hirohisa", " The Musket Room", " Rebelle", " Uchu", " Uncle Boons", " Contra", " B\u00e2tard", " Jungsik", " Delaware and Hu

In [15]:
with open('./Resources/coordinates.json', 'w', encoding='utf-8') as f:
    json.dump(data_set, f, ensure_ascii=False, indent=4)

In [16]:
from pprint import pprint

json_dump

'{"restaurants": [[" Sushi Inoue", " Dovetail", " Cafe Boulud", " Jean-Georges", " Masa", " Marea", " Daniel", " Torishin", " Le Bernardin", " The Modern", " Aquavit", " Caviar Russe", " Satsuki (Suzuki\\u2019s sushi bar)", " Chef\'s Table at Brooklyn Fare", " Aureole", " Gabriel Kreuther", " Sushi Ginza Onodera", " Sushi Amane", " Sushi Yasuda", " Ai Fiori", " Caf\\u00e9 China", " Kajitsu", " Tempura Matsui", " Ushiwakamaru", " The Breslin", " The NoMad Restaurant", " Del Posto", " Casa Enrique", " Junoon", " La Sirena", " Eleven Madison Park", " The Clocktower", " Cote Korean Steakhouse", " Rouge Tomate Chelsea", " Gunter Seeger NY", " ALDEA", " Gramercy Tavern", " Casa Mono", " Walls\\u00e9", " Gotham Bar and Grill", " Nix", " Babbo", " Blue Hill", " Kanoyama", " Minetta Tavern", " Sushi Zo", " Carbone", " ZZ\'s Clam Bar", " Jewel Bako", " Kyo Ya", " Momofuku Ko", " Hirohisa", " The Musket Room", " Rebelle", " Uchu", " Uncle Boons", " Contra", " B\\u00e2tard", " Jungsik", " Delaware

In [20]:
stars = []

for i in range(len(restaurants)):
    star = michelin_df['star'][i]
    stars.append(star)

# stars

In [24]:
for i in range(len(restaurants)):
    print("{")
    print(f'"formatted_address" : "{address[i]}"'+",")
    print(f'"geometry" : '+ "{")
    print(f'  "location" :'+ "{")
    print(f'     "lat" : {lat[i]}' + ",")
    print(f'     "lon" : {lon[i]}')
    print("    } ")
    print("  }, ")
#     print(f'"michelin_stars" : "{stars[i]}"'+",")
    print(f'"name" : "{names[i]}"'+",")
    print("}, ")
    print(" ")


{
"formatted_address" : "381 Lenox Ave, New York, NY 10027",
"geometry" : {
  "location" :{
     "lat" : 40.7313524,
     "lon" : -74.0065048
    } 
  }, 
"name" : " Sushi Inoue",
}, 
 
{
"formatted_address" : "103 W 77th St, New York, NY 10024",
"geometry" : {
  "location" :{
     "lat" : 40.70544486444615,
     "lon" : -73.99429321289062
    } 
  }, 
"name" : " Dovetail",
}, 
 
{
"formatted_address" : "20 E 76th St, New York, NY 10021",
"geometry" : {
  "location" :{
     "lat" : 40.7136187,
     "lon" : -73.9888503
    } 
  }, 
"name" : " Cafe Boulud",
}, 
 
{
"formatted_address" : "1 Central Park West, New York, NY 10023",
"geometry" : {
  "location" :{
     "lat" : 40.7327,
     "lon" : -73.99766
    } 
  }, 
"name" : " Jean-Georges",
}, 
 
{
"formatted_address" : "10 Columbus Cir, New York, NY 10019",
"geometry" : {
  "location" :{
     "lat" : 40.75851,
     "lon" : -73.98322
    } 
  }, 
"name" : " Masa",
}, 
 
{
"formatted_address" : "240 Central Park S, New York, NY 10019",
"

In [ ]:
"results" : [
      {
         "formatted_address" : "1 Central Park West, New York, NY 10023, United States",
         "geometry" : {
            "location" : {
               "lat" : 40.769084,
               "lng" : -73.981431
            }
         },

         "name" : "Jean-Georges",
      },